In [ ]:

import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
import os
if os.getcwd().endswith('ALPACA-paper'):
    os.chdir('bin/ANALYSIS')

num_exps = 25000

filepath = "../../_assets/tumour_growth/exome_8Aug_nosel/"
base_filepath = "../../_assets/tumour_growth/"


def assemble_data(num_exps, title):
    snv_rates, cna_rates, results = np.zeros(num_exps), np.zeros(num_exps), np.zeros(num_exps)
    for n in range(1, num_exps+1):
        [snv_rate, cna_rate, res] = np.load(filepath+"res_with_params_"+str(n)+".npy", allow_pickle=True)
        snv_rates[n-1] = snv_rate
        cna_rates[n-1] = cna_rate
        results[n-1] = res
    np.save("../../_assets/tumour_growth/CNA_snv_rates"+title+".npy", snv_rates)
    np.save("../../_assets/tumour_growth/CNA_cna_rates"+title+".npy", cna_rates)
    np.save("../../_assets/tumour_growth/CNA_results"+title+".npy", results)

def load_cna_results(title):
    snv_rates = np.load("../../_assets/tumour_growth/CNA_snv_rates"+title+".npy", allow_pickle=True)
    cna_rates = np.load("../../_assets/tumour_growth/CNA_cna_rates"+title+".npy", allow_pickle=True)
    results = np.load("../../_assets/tumour_growth/CNA_results"+title+".npy", allow_pickle=True)
    return snv_rates, cna_rates, results

def get_av_matrix(snv_rates, cna_rates, results, size=50):
    frac_cnas = cna_rates/snv_rates 
    uniq_snvs = np.unique(snv_rates)
    av_matrix = np.zeros((size, size))
    for n, snv in enumerate(uniq_snvs):
        locs_1 = np.where(snv_rates==snv)[0]
        cnas_here = np.unique(frac_cnas[locs_1])
        #print(cnas_here)
        for m, cna in enumerate(cnas_here):
            locs_2 = np.where(frac_cnas==cna)[0]
            relevant = np.intersect1d(locs_1, locs_2)
            av_matrix[n][m] = np.average(results[relevant])
    return uniq_snvs, av_matrix

def plot_av_matrix(av_matrix, title, use_colorbar=False):
    fig, ax = plt.subplots(figsize=(8, 8))
    im = ax.matshow(av_matrix.T, origin='lower', cmap=mpl.cm.seismic, vmin=0, vmax=1)
    ax.set_xticks(np.linspace(0, 49, num=9), np.around(np.linspace(0, 0.8, num=9), decimals=1))
    ax.set_yticks(np.linspace(0, 49, num=9), np.around(np.linspace(0, 0.8, num=9), decimals=1))
    ax.set_xlabel("SNV mutation rate, per cell division")
    ax.set_ylabel("CNA mutation rate, per cell division")
    savetitle = base_filepath+"/cna_figs/"+title
    if use_colorbar:
        cbar = plt.colorbar(im)
        cbar.ax.set_ylabel("Clustering consistency", fontsize=20)
        savetitle += "_with_cbar"
    plt.title(title)
    plt.show()
    fig.savefig(savetitle+".png", dpi=300)
    fig.savefig(savetitle+".pdf", format="pdf", dpi=300)
    figs_dir_title = "../../figures/"+"Suppfig1a_b_c"+title.replace(" ", "_")
    fig.savefig(figs_dir_title+".pdf", format="pdf", dpi=300)
    plt.clf()


In [ ]:


plt.rcParams['font.family'] = 'arial'
plt.rcParams['font.size'] = 20

assemble_data(num_exps, "")
snv_rates, cna_rates, results = load_cna_results("")
uniq_snvs, av_matrix = get_av_matrix(snv_rates, cna_rates, results)
#np.save("../../_assets/tumour_growth/CNA_uniq_snvs.npy", uniq_snvs)
#np.save("../../_assets/tumour_growth/CNA_uniq_frac_cnas.npy", uniq_frac_cnas)
#np.save("../../_assets/tumour_growth/CNA_av_matrix.npy", av_matrix)
plot_av_matrix(av_matrix, "With no selection")
plot_av_matrix(av_matrix, "With no selection", use_colorbar=True)


    




In [ ]:
filepath = "../../_assets/tumour_growth/exome_8Aug_sel_CNA/"




assemble_data(num_exps, "_CNA_only")
snv_rates, cna_rates, results = load_cna_results("_CNA_only")
uniq_snvs, av_matrix = get_av_matrix(snv_rates, cna_rates, results)
np.save("../../_assets/tumour_growth/CNA_uniq_snvs_CNA_only.npy", uniq_snvs)
#np.save("../../_assets/tumour_growth/CNA_uniq_frac_cnas.npy", uniq_frac_cnas)
np.save("../../_assets/tumour_growth/CNA_av_matrix_CNA_only.npy", av_matrix)
plot_av_matrix(av_matrix, "With selection on CNAs only", use_colorbar=True)
plot_av_matrix(av_matrix, "With selection on CNAs only")


In [ ]:
filepath = "../../_assets/tumour_growth/exome_8Aug_sel_all/"



assemble_data(num_exps, "_all_sel")
snv_rates, cna_rates, results = load_cna_results("_all_sel")
uniq_snvs, av_matrix = get_av_matrix(snv_rates, cna_rates, results)
np.save("../../_assets/tumour_growth/CNA_uniq_snvs_all_sel.npy", uniq_snvs)
#np.save("../../_assets/tumour_growth/CNA_uniq_frac_cnas.npy", uniq_frac_cnas)
np.save("../../_assets/tumour_growth/CNA_av_matrix_all_sel.npy", av_matrix)
plot_av_matrix(av_matrix, "With selection on all mutations")
plot_av_matrix(av_matrix, "With selection on all mutations", use_colorbar=True)


In [ ]:
#just a colorbar on its own

fig, ax = plt.subplots(figsize=(6, 1), layout='constrained')

cmap = mpl.cm.seismic
norm = mpl.colors.Normalize(vmin=0, vmax=1)

fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
             cax=ax, orientation='horizontal', label="clustering consistency")

fig.savefig(base_filepath+"/cna_figs/cbar.png", dpi=300)

fig.savefig(base_filepath+"/cna_figs/cbar.pdf", format="pdf", dpi=300)
plt.show()

plt.clf()